In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr


from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
#!pip install sklearn-xarray

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from math import radians, sin, cos, sqrt, atan2

In [ ]:
DataSet = xr.open_dataset("/content/drive/MyDrive/BTTAI_Project_Team_B/data_set_files/final_merged_dataset.nc")
DataSet

<xarray.Dataset>
Dimensions:        (lon: 600, lat: 258, year: 95)
Coordinates:
  * lon            (lon) float64 -171.9 -171.6 -171.4 ... -22.62 -22.38 -22.12
  * lat            (lat) float64 12.12 12.38 12.62 12.88 ... 75.88 76.12 76.38
  * year           (year) int32 2006 2007 2008 2009 2010 ... 2097 2098 2099 2100
Data variables:
    null_variable  (lat, lon, year) object ...
    maxHuss        (year, lat, lon) float32 ...
    maxPrec        (year, lat, lon) float32 ...
    maxTemp        (year, lat, lon) float32 ...
    meanHuss       (year, lat, lon) float32 ...
    meanPrec       (year, lat, lon) float32 ...
    meanRsds       (year, lat, lon) float32 ...
    meanTemp       (year, lat, lon) float32 ...
    minHuss        (year, lat, lon) float32 ...
    minPrec        (year, lat, lon) float32 ...
    minTemp        (year, lat, lon) float32 ...

In [ ]:

subset_da = DataSet.sel(lat=slice(24.396308, 49.384358), lon=slice(-124.848974, -66.885444),year=slice(2006,2023))
df = subset_da.to_dataframe()
df.reset_index(inplace=True)
df.dropna(subset=['maxHuss'], inplace=True)
df = df.replace(to_replace='', value=np.nan)
#df = df.dropna(subset='null_variable')


df['wns'] = df['null_variable'].map(lambda x: 1 if x == 'W' else 0)

# Select instances where the label is 0
zero_label_indices = df[df['wns'] == 0].index

# Randomly select a subset of instances with label 0
# For example, let's select 30% of instances with label 0
subset_size = int(0.998 * len(zero_label_indices))
random_subset_indices = np.random.choice(zero_label_indices, size=subset_size, replace=False)

# Update the data by keeping only the randomly selected instances
selected_data = df.drop(random_subset_indices)


# Update the data by keeping only the randomly selected instances
selected_data.drop('null_variable', axis=1, inplace=True)
df = selected_data
df

,lon,lat,year,maxHuss,maxPrec,maxTemp,meanHuss,meanPrec,meanRsds,meanTemp,minHuss,minPrec,minTemp,wns
4993,-124.125,43.875,2013,0.011750,67.224129,23.927216,0.006848,5.924083,146.402512,11.236343,0.003607,0.0,-1.331177,0
8546,-123.625,43.125,2020,0.010284,44.469723,29.811646,0.005530,3.488417,168.499725,11.169211,0.001707,0.0,-6.809692,0
8636,-123.625,44.375,2020,0.011677,65.337646,30.905121,0.006119,5.232748,156.860794,11.132643,0.002514,0.0,-7.150970,0
8661,-123.625,44.875,2009,0.010531,71.109344,25.242462,0.005620,8.322410,142.492432,9.486574,0.001837,0.0,-9.232239,0
8872,-123.625,47.625,2022,0.010778,102.644287,29.025055,0.005777,9.181305,141.962585,7.963105,0.001768,0.0,-9.206238,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406444,-68.375,44.625,2010,0.014918,43.269913,28.719238,0.005610,3.394894,163.678421,8.533190,0.000533,0.0,-20.052628,1
406445,-68.375,44.625,2011,0.014342,65.899597,26.864532,0.005432,4.252778,156.201004,9.602814,0.000520,0.0,-17.554596,1
412018,-67.625,46.875,2022,0.015262,57.338802,28.974487,0.004860,3.599618,158.328354,5.838346,0.000275,0.0,-20.577988,0
413819,-67.375,46.875,2023,0.015964,44.231045,25.947388,0.004683,4.063868,160.951614,5.242080,0.000272,0.0,-20.063034,0


In [ ]:



# Sort DataFrame by Year in ascending order
df = df.sort_values(by='year')

# Function to check if WNS was detected in previous years for a specific coordinate
def wns_detected_past_years(row):
    past_years_data = df[(df['lat'] == row['lat']) & (df['lon'] == row['lon']) & (df['year'] < row['year'])]
    if len(past_years_data) > 0 and (past_years_data['wns'] == 1).any():
        return 1  # WNS detected in past years for this coordinate
    else:
        return 0  # WNS not detected in past years for this coordinate

# Apply the function to create a new column 'WNS_Detected_Past_Years'
df['WNS_Detected_Past_Years'] = df.apply(wns_detected_past_years, axis=1)


In [ ]:
# Function to calculate distance between two coordinates using Haversine formula


def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Function to calculate shortest distance for each point to a coordinate with previous WNS detection
def shortest_distance_to_wns(row):
    wns_detected_coords = df[(df['WNS_Detected_Past_Years'] == 1)]
    distances = [haversine(row['lat'], row['lon'], lat, lon) for lat, lon in wns_detected_coords[['lat', 'lon']].values]
    if len(distances) > 0:
        return min(distances)
    else:
        return None

# Apply the function to create a new column 'Shortest_Distance_to_WNS'
df['Shortest_Distance_to_WNS'] = df.apply(shortest_distance_to_wns, axis=1)

#after inspecting df it looks weird..


In [ ]:


# Assuming 'df' is your DataFrame containing spatiotemporal data
# X contains your features, y contains the target variable ('WNS')
# Modify this based on your actual column names
X = df[['lon', 'lat', 'year', 'maxHuss', 'maxPrec', 'maxTemp', 'meanHuss', 'meanPrec', 'meanRsds', 'meanTemp', 'minHuss', 'minTemp','Shortest_Distance_to_WNS']]
y = df['wns']

# Split the data into training and test sets (maintaining temporal order if required)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the data into training and testing sets based on the cutoff year
cutoff_year = 2020

X_train = X[df['year'] < cutoff_year]
y_train = y[df['year'] < cutoff_year]
X_test = X[df['year'] >= cutoff_year]
y_test = y[df['year'] >= cutoff_year]

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Additional evaluation metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.85
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.99      0.92       140
           1       0.86      0.20      0.32        30

    accuracy                           0.85       170
   macro avg       0.85      0.60      0.62       170
weighted avg       0.85      0.85      0.81       170



In [ ]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

# Save the trained Random Forest model to a file using pickle
with open('random_forest_model_2024.pkl', 'wb') as file:
    pickle.dump(rf_classifier, file)

In [ ]:
from google.colab import files
files.download('random_forest_model_2024.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>